**Bike Network Data Wrangling**

This script takes the .csv files produced in script 01-bike-network-metric-generation.ipynb and munges the data to create files that are easier to work with.

In [1]:
# libary imports
import geopandas as gpd
import pandas as pd
import os
import glob

# get working directory
working_directory =  os.getcwd()


In [2]:
# get a list of all locations output from script 01

globbed_files = glob.glob(working_directory + "\data\\places\\*.csv" )

# set loop counter
counter = 1

# transpose and join all csv files

for csv in globbed_files:
    metrics = pd.read_csv(csv) # read in csv
    metrics['place'] = os.path.basename(csv) # get place name
    metrics['place'] = metrics['place'].str.replace('.csv','') # remove spare data
    metrics.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore') # remove spare columns 
    place = metrics.at[0,'place'] # save place as variable
    metrics.loc[len(metrics.index)] = ['place', place, place, place] # add place as row
    metrics.drop('place', axis=1, inplace=True) # remove unneeded columns 
    metrics.drop([6,7], axis=0, inplace=True) # remove unneeded rows
    metrics.set_index('metric') 

    # create base dataframe for other files to joined to
    if counter == 1:
        a = metrics
        a = a.T
        a.columns = a.iloc[0]
        a = a.iloc[1:] # remove spare row
        a = a.T
        a_headers = a.columns.tolist()
        a_place = a.loc['place','bike_values']
        a.columns = ['bike_values ' + a_place, 'road_values ' + a_place] # move from having two rows to one combined row
        counter = counter + 1

    else:
        b = metrics
        b = b.T  
        b.columns = b.iloc[0]
        b = b.iloc[1:]
        b = b.T
        b_headers = b.columns.tolist()
        b_place = b.loc['place','bike_values']
        b.columns = ['bike_values ' + b_place, 'road_values ' + b_place]
        c = a.join(b, how = 'outer').drop_duplicates() # join dataframes together
        a = c  # reset dataframe a
        counter = counter + 1 


C:\Users\b8008458\AppData\Local\Temp\ipykernel_23884\4237037577.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  metrics['place'] = metrics['place'].str.replace('.csv','') # remove spare data
C:\Users\b8008458\AppData\Local\Temp\ipykernel_23884\4237037577.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  metrics['place'] = metrics['place'].str.replace('.csv','') # remove spare data
C:\Users\b8008458\AppData\Local\Temp\ipykernel_23884\4237037577.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  metrics['place'] = metrics['place'].str.replace('.csv','') # remove spare data
C:\Users\b8008458\AppData\Local\Temp\ipykernel_23884\4237037577.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  metrics['place'] = metrics['place'].str.replace('.csv','') # remove spare data
C:\Users\b800845

In [3]:
# transpose dataframe
c = c.T

# split dataframe
bike_df = c.iloc[::2, :]
road_df = c.iloc[1::2, :]

# change column headings before rejoining
road_df.columns = [str(col) + '_road' for col in road_df.columns]
bike_df.columns = [str(col) + '_bike' for col in bike_df.columns]

# set temp index
bike_df.insert(0, 'temp_index', range(0, len(bike_df)))
road_df.insert(0, 'temp_index', range(0, len(road_df)))

# merge and set place as index
combined_df = bike_df.merge(road_df, on = 'temp_index')
combined_df = combined_df.set_index('place_bike')

combined_df = combined_df.drop('temp_index', axis = 1)
combined_df = combined_df.drop('place_road', axis = 1)

# reset axis name
combined_df = combined_df.rename_axis('place')

# clean dataframe
combined_df = combined_df.drop(['0way_int_count_bike', '0way_int_prop_bike'], axis=1, errors='ignore')

# clean bike dataframe
bike_df = bike_df.drop('temp_index', axis = 1)
bike_df = bike_df.set_index('place_bike')
bike_df = bike_df.rename_axis('place')
bike_df = bike_df.drop(['0way_int_count_bike', '0way_int_prop_bike'], axis=1, errors='ignore')

# clean road dataframe
road_df = road_df.drop('temp_index', axis = 1)
road_df = road_df.set_index('place_road')
road_df = road_df.rename_axis('place')
road_df = road_df.drop(['0way_int_count_bike', '0way_int_prop_bike'], axis=1, errors='ignore')

In [4]:
# save to csv file

combined_df.to_csv(working_directory + "\Data\\Cleaned_Data\\combined_places.csv")

bike_df.to_csv(working_directory + "\Data\\Cleaned_Data\\bike_place.csv")

road_df.to_csv(working_directory + "\Data\\Cleaned_Data\\road_places.csv")
